In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import joblib

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from mapchiral import mapchiral

from tqdm import tqdm
tqdm.pandas()

import tmap as tm
from faerun import Faerun

plt.rcParams['font.sans-serif'] = "Menlo"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams.update({'font.size': 20})

#### Helper Functions

In [2]:
cmap = ListedColormap(['#BCCFFF', '#648FFF', '#0042EF', '#D688AE', '#DC267F', '#A90052'])

In [3]:
def plot_tmap(query_name, savename):

    # Extract the data
    csv_files = [filename for filename in os.listdir('results/') if f'{query_name}' in filename and filename.endswith('.csv')]
    csv_files = sorted(csv_files, key=lambda x: int(x.split('_')[3].split('.')[0]))

    # Combine data from runs
    df = pd.DataFrame()
    for i, csv_file in enumerate(csv_files):
        df_run = pd.read_csv(f'results/{csv_file}').head(1000)
        df_run['run'] = int(csv_files[i].split('_')[3].split('.')[0]) + 1
        df_run['fingerprint'] = csv_files[i].split('_')[2]
        df = pd.concat([df, df_run]).reset_index(drop=True)
    
    # Clean data frame
    df.sort_values(by=['fingerprint', 'generation'], inplace=True)
    df['map4c'] = df['smiles'].apply(lambda x: mapchiral.encode(Chem.MolFromSmiles(x), max_radius=2, n_permutations=2048))
    df['set'] = df['fingerprint'] + ': ' + df['run'].astype(str)

    # Calculate TMAP layout 
    if not os.path.exists(f'tmaps/{savename}_tmap_layout.pkl'):
        lf = tm.LSHForest(512, 32)
        map4 = np.array(df['map4c'])
        fps = []
        for i in map4:
            vec = tm.VectorUint(i)
            fps.append(vec)
        lf.batch_add(fps)
        lf.index()
        cfg = tm.LayoutConfiguration()
        cfg.node_size = 1/32 
        cfg.mmm_repeats = 2 
        cfg.sl_extra_scaling_steps = 5 
        cfg.k = 2000 
        cfg.sl_scaling_type = tm.RelativeToAvgLength 
        x, y, s, t, _ = tm.layout_from_lsh_forest(lf, cfg)
        tm_layout = {'x': list(x), 'y': list(y), 's': list(s), 't': list(t)}
        joblib.dump(tm_layout, f'tmaps/{savename}_tmap_layout.pkl')
    else:
        tm_layout = joblib.load(f'tmaps/{savename}_tmap_layout.pkl')

    # Define categories
    set_labels, set_data = Faerun.create_categories(df['set'])

    # Plot TMAP
    f = Faerun(view="front", coords=False, title="", clear_color="#FFFFFF", thumbnail_width=500)
    f.add_scatter(f'{query_name}',{"x": tm.VectorFloat(tm_layout['x']), "y": tm.VectorFloat(tm_layout['y'] ), "c": [set_data], "labels": df['smiles'].values.tolist()}, shader="sphere", point_scale=5, max_point_size=20, legend_labels=[set_labels], categorical=[True], colormap=[cmap], series_title=['Set'], has_legend=True)
    f.add_tree(f'{query_name}_tree', {"from": tm.VectorUint(tm_layout['s']), "to": tm.VectorUint(tm_layout['t'])}, point_helper=f'{query_name}')
    f.plot(f'tmaps/{savename}_tmap', template='smiles')

Epothilone A

In [4]:
plot_tmap('EpothiloneA', 'epothilone')

/home/markus/Developer/Code/10E60/tmaps/epothilone_tmap.html

Cyclodextrin

In [5]:
plot_tmap('aCyclodextrin', 'cyclodextrin')

/home/markus/Developer/Code/10E60/tmaps/cyclodextrin_tmap.html

Cyclosporin

In [6]:
plot_tmap('Cyclosporin', 'cyclosporin')

/home/markus/Developer/Code/10E60/tmaps/cyclosporin_tmap.html

Nonactin

In [7]:
plot_tmap('Nonactin', 'nonactin')

/home/markus/Developer/Code/10E60/tmaps/nonactin_tmap.html

Onchidin

In [8]:
plot_tmap('Onchidin', 'onchidin')

/home/markus/Developer/Code/10E60/tmaps/onchidin_tmap.html

Valinomycin

In [9]:
plot_tmap('Valinomycin', 'valinomycin')

/home/markus/Developer/Code/10E60/tmaps/valinomycin_tmap.html

#### TMAP of random + self runs

Import random runs

In [10]:
csv_files = [filename for filename in os.listdir('results/') if f'PolymyxinB2_MAP4' in filename and 'GramicidinS' not in filename and 'PolymyxinB2_PolymyxinB2' not in filename and filename.endswith('.csv')]
csv_files = sorted(csv_files, key=lambda x: ['1', '0', '2'].index(x.split('_')[3].split('.')[0]))

df_rnd = pd.DataFrame()
for i, csv_file in enumerate(csv_files):
    df_run = pd.read_csv(f'results/{csv_file}')
    df_run = df_run[df_run['dist'] < 0.5]
    df_run['run'] = i 
    df_rnd = pd.concat([df_rnd, df_run])

df_rnd.dropna(inplace=True)
df_rnd.sort_values(by=['run', 'generation'], inplace=True)
df_rnd['set'] = 'Random'

Import self run (since they all generate the same compounds)

In [11]:
df_self = pd.read_csv('results/20240523_PolymyxinB2_PolymyxinB2_MAP4_0.csv')
df_self = df_self[df_self['dist'] < 0.5]
df_self['run'] = 0
df_self.dropna(inplace=True)
df_self.sort_values(by=['run', 'generation'], inplace=True)
df_self['set'] = 'Self'

Combine

In [12]:
df = pd.concat([df_rnd, df_self]).reset_index(drop=True)
df['run'] = df['run'] + 1
df['set_combined'] = df['set'] + ': ' + df['run'].astype(str)
df.drop_duplicates(subset=['smiles'], inplace=True)

Determine Levenshtein distance to Polymyxin B2

In [13]:
def levenshtein_distance(seq1, seq2):
    blocks1 = seq1.split('-')
    blocks2 = seq2.split('-')
    len1 = len(blocks1)
    len2 = len(blocks2)

    dp = [[0] * (len2 + 1) for _ in range(len1 + 1)]
    
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j
    
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            if blocks1[i - 1] == blocks2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1
    
    if "c" in seq1 and "c" in seq2:
        return dp[len1][len2]
    elif "c" in seq1 and "c" not in seq2:
        return dp[len1][len2] - 1
    elif "c" not in seq1 and "c" in seq2:
        return dp[len1][len2] - 1
    else:
        return dp[len1][len2]
    
polymyxin_sequence = 'T011-BB045-BB012-BB045-b000-BB045-BB030-BB003-BB045-BB045-BB012'
df['levenshtein'] = df['sequence'].apply(lambda x: levenshtein_distance(x, polymyxin_sequence))

Calculate fingerprints

In [14]:
df['map4c'] = df['smiles'].progress_apply(lambda x: mapchiral.encode(Chem.MolFromSmiles(x), max_radius=2, n_permutations=2048))

100%|██████████| 12966/12966 [05:25<00:00, 39.83it/s]


Calculate layout

In [15]:
if not os.path.exists(f'tmaps/polymyxin_randself_tmap_layout.pkl'):
    lf = tm.LSHForest(512, 32)
    map4 = np.array(df['map4c'])
    fps = []
    for i in map4:
        vec = tm.VectorUint(i)
        fps.append(vec)
    lf.batch_add(fps)
    lf.index()
    cfg = tm.LayoutConfiguration()
    cfg.node_size = 1/32 
    cfg.mmm_repeats = 2 
    cfg.sl_extra_scaling_steps = 5 
    cfg.k = 2000 
    cfg.sl_scaling_type = tm.RelativeToAvgLength 
    x, y, s, t, _ = tm.layout_from_lsh_forest(lf, cfg)
    tm_layout = {'x': list(x), 'y': list(y), 's': list(s), 't': list(t)}
    joblib.dump(tm_layout, f'tmaps/polymyxin_randself_tmap_layout.pkl')
else:
    tm_layout = joblib.load(f'tmaps/polymyxin_randself_tmap_layout.pkl')

Plot

In [16]:
df['highlights'] = ''
df.loc[df['smiles'] == 'CC(C)CCCCC(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H]([C@@H](C)O)C(=O)N[C@@H]([C@H](C)O)C(=O)N[C@@H](CCN8)C(=O)N[C@@H](CC(C)C)C(=O)N[C@H](CC1=CC=CC=C1)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CC[NH3+])C(=O)N[C@H](CC[NH3+])C(=O)N[C@@H]([C@@H](C)O)C(=O)8', 'highlights'] = '7'
df.loc[df['smiles'] == 'N9[C@@H](CC[NH3+])C(=O)N[C@@H](CC(C)C)C(=O)N[C@H](CC1=CC=CC=C1)C(=O)N[C@@H](CC1CCCCC1)C(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H]([C@H](C)O)C(=O)NCCCC(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H]([C@H](C)O)C(=O)N[C@@H](CC[NH3+])C(=O)9', 'highlights'] = '8'
df.loc[df['smiles'] == 'CC(C)CCCCC(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H]([C@H](C)O)C(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H](CCN8)C(=O)N[C@@H](CC[NH3+])C(=O)N[C@H](CC1=CC=CC=C1)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H](CC[NH3+])C(=O)N[C@@H]([C@H](C)O)C(=O)8', 'highlights'] = 'PMB2'

In [18]:
cmap = ListedColormap(['#648FFF', '#DC267F', '#FFB000', '#CECECE'])
set_labels, set_data = Faerun.create_categories(df['set_combined'])
hihglight_labels, highlight_data = Faerun.create_categories(df['highlights'])

# Plot TMAP
f = Faerun(view="front", coords=False, title="", clear_color="#FFFFFF", thumbnail_width=500)
f.add_scatter(f'polymyxin_randself',
              {"x": tm.VectorFloat(tm_layout['x']),
                "y": tm.VectorFloat(tm_layout['y'] ), 
                "c": [set_data, 
                      df['dist'].values.tolist(),
                      df['levenshtein'].values.tolist()],
                "labels": df['smiles'].values.tolist()
                }, 
            shader="sphere", 
            point_scale=4, 
            max_point_size=20, 
            legend_labels=[set_labels, None, None], 
            categorical=[True, False, False], 
            colormap=[cmap, 'turbo', 'turbo'], 
            series_title=['Set', 'Jaccard Distance', 'Levenshtein Distance'], 
            has_legend=True)
f.add_tree(f'polymyxin_randself_tree', {"from": tm.VectorUint(tm_layout['s']), "to": tm.VectorUint(tm_layout['t'])}, point_helper=f'polymyxin_randself')
f.plot(f'tmaps/polymyxin_randself_tmap', template='smiles')

/home/markus/Developer/Code/10E60/tmaps/polymyxin_randself_tmap.html